In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
import os
import time
import copy

# Define data directories
data_dir = "/kaggle/input/plantvillage-traditional-augmentation/PlantVillage"
train_dir = data_dir + "/kaggle/input/plantvillage-traditional-augmentation/PlantVillage/train"
val_dir = data_dir + "/kaggle/input/plantvillage-traditional-augmentation/PlantVillage/val"

# Define transformation for the data
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Load the datasets with ImageFolder
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
                  for x in ['train', 'val']}

# Define the dataloaders
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=4)
               for x in ['train', 'val']}

# Number of classes in the dataset
num_classes = 38

# Load the pre-trained VGG16 model
model = models.vgg16(pretrained=True)
# Change the last fully connected layer to match the number of classes

# Freeze all layers except the final fully connected layer
for param in model.parameters():
    param.requires_grad = False

# Modify the final fully connected layer to match the number of classes
model.classifier[6] = nn.Linear(4096, num_classes)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# Function to train the model
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward
                # Track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / len(image_datasets[phase])
            epoch_acc = running_corrects.double().to(torch.float32) / len(image_datasets[phase])

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # Deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    
    # Load best model weights
    model.load_state_dict(best_model_wts)
    return model

# Train the model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

model = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=25)

# Save the trained model
torch.save(model.state_dict(), 'custom_vgg16.pth')


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:03<00:00, 179MB/s] 


Epoch 0/24
----------
train Loss: 1.2758 Acc: 0.6535
val Loss: 0.6899 Acc: 0.8161

Epoch 1/24
----------
train Loss: 1.1753 Acc: 0.7001
val Loss: 0.6038 Acc: 0.8389

Epoch 2/24
----------
train Loss: 1.1645 Acc: 0.7123
val Loss: 0.5339 Acc: 0.8509

Epoch 3/24
----------
train Loss: 1.1538 Acc: 0.7170
val Loss: 0.5030 Acc: 0.8625

Epoch 4/24
----------
train Loss: 1.1670 Acc: 0.7211
val Loss: 0.6477 Acc: 0.8461

Epoch 5/24
----------
train Loss: 1.1379 Acc: 0.7269
val Loss: 0.5430 Acc: 0.8563

Epoch 6/24
----------
train Loss: 1.1433 Acc: 0.7285
val Loss: 0.5089 Acc: 0.8650

Epoch 7/24
----------
train Loss: 0.9799 Acc: 0.7503
val Loss: 0.3729 Acc: 0.8920

Epoch 8/24
----------
train Loss: 0.9044 Acc: 0.7614
val Loss: 0.3451 Acc: 0.8978

Epoch 9/24
----------
train Loss: 0.8527 Acc: 0.7704
val Loss: 0.3489 Acc: 0.8976

Epoch 10/24
----------
train Loss: 0.8199 Acc: 0.7726
val Loss: 0.3478 Acc: 0.8941

Epoch 11/24
----------
train Loss: 0.8151 Acc: 0.7757
val Loss: 0.3209 Acc: 0.9027

Ep